# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:

# !pip install hvplot
# !pip install cftime
# !pip install geoviews

  Using cached geoviews-1.9.6-py2.py3-none-any.whl (508 kB)
     ---------------------------------------- 1.4/1.4 MB 2.9 MB/s eta 0:00:00
  Using cached Cartopy-0.21.1.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 5.1/5.1 MB 3.4 MB/s eta 0:00:00
  Using cached pyshp-2.3.1-py2.py3-none-any.whl (46 kB)
Failed to build cartopy


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [286 lines of output]
  <string>:90: UserWarning: Unable to determine GEOS version. Ensure you have 3.7.2 or later installed, or installation may fail.
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\cartopy
  copying lib\cartopy\crs.py -> build\lib.win-amd64-cpython-310\cartopy
  copying lib\cartopy\geodesic.py -> build\lib.win-amd64-cpython-310\cartopy
  copying lib\cartopy\img_transform.py -> build\lib.win-amd64-cpython-310\cartopy
  copying lib\cartopy\util.py -> build\lib.win-amd64-cpython-310\cartopy
  copying lib\cartopy\vector_transform.py -> build\lib.win-amd64-cpython-310\cartopy
  copying lib\cartopy\_epsg.py -> build\lib.win-amd64-cpython-310\cartopy
  copying lib\cartopy\_version.py -> build\lib.win-amd64-cpython-310\cartopy
  copying lib\cartopy\__init__.py -> build\lib.win-amd64-cpython-310\cartopy
  

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
import hvplot.pandas
import geoviews as gv
import cartopy.crs as ccrs

# Import API key
from api_keys import geoapify_key


In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID         City Country  Latitude  Longitude  Max Temperature  \
0        0    hithadhoo      MV   -0.6000    73.0833            28.81   
1        1  barentsburg     NaN       NaN        NaN              NaN   
2        2       atuona      PF   -9.8000  -139.0333            27.38   
3        3    rovaniemi      FI   66.5000    25.7167            -2.33   
4        4   oudtshoorn      ZA  -33.5907    22.2014            13.79   

   Humidity (%)  Cloudiness  Wind Speed  Datetime (Local)  
0          69.0        90.0        3.07      1.681604e+09  
1           NaN         NaN         NaN               NaN  
2          74.0         6.0        6.04      1.681604e+09  
3          43.0         0.0        2.06      1.681604e+09  
4          56.0         6.0        0.65      1.681604e+09

In [10]:
print(city_data_df.count())


City_ID             640
City                640
Country             594
Latitude            598
Longitude           598
Max Temperature     598
Humidity (%)        598
Cloudiness          598
Wind Speed          598
Datetime (Local)    598
dtype: int64


In [11]:
city_data_df=city_data_df.dropna(how="any")

city_data_df.head()

City_ID        City Country  Latitude  Longitude  Max Temperature  \
0        0   hithadhoo      MV   -0.6000    73.0833            28.81   
2        2      atuona      PF   -9.8000  -139.0333            27.38   
3        3   rovaniemi      FI   66.5000    25.7167            -2.33   
4        4  oudtshoorn      ZA  -33.5907    22.2014            13.79   
5        5   tuatapere      NZ  -46.1333   167.6833            17.87   

   Humidity (%)  Cloudiness  Wind Speed  Datetime (Local)  
0          69.0        90.0        3.07      1.681604e+09  
2          74.0         6.0        6.04      1.681604e+09  
3          43.0         0.0        2.06      1.681604e+09  
4          56.0         6.0        0.65      1.681604e+09  
5          55.0        98.0        1.79      1.681604e+09

In [12]:
print(city_data_df.count())



City_ID             594
City                594
Country             594
Latitude            594
Longitude           594
Max Temperature     594
Humidity (%)        594
Cloudiness          594
Wind Speed          594
Datetime (Local)    594
dtype: int64


In [13]:
city_data_df.dtypes

City_ID               int64
City                 object
Country              object
Latitude            float64
Longitude           float64
Max Temperature     float64
Humidity (%)        float64
Cloudiness          float64
Wind Speed          float64
Datetime (Local)    float64
dtype: object

In [14]:
city_data_df.columns

Index(['City_ID', 'City', 'Country', 'Latitude', 'Longitude',
       'Max Temperature', 'Humidity (%)', 'Cloudiness', 'Wind Speed',
       'Datetime (Local)'],
      dtype='object')

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
# %%capture --no-display
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity (%)",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (%))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_condition=city_data_df.copy()
max_temp = (ideal_weather_condition["Max Temperature"] <= 27) & (ideal_weather_condition["Max Temperature"] > 21)
wind_speed = ideal_weather_condition["Wind Speed"] <4.5
cloudiness = ideal_weather_condition["Cloudiness"] == 0

ideal_weather_condition[max_temp & wind_speed & cloudiness]
# Drop any rows with null values
ideal_weather_condition=ideal_weather_condition.dropna(how="any")

# Display sample data
ideal_weather_condition.head()

City_ID        City Country  Latitude  Longitude  Max Temperature  \
0        0   hithadhoo      MV   -0.6000    73.0833            28.81   
2        2      atuona      PF   -9.8000  -139.0333            27.38   
3        3   rovaniemi      FI   66.5000    25.7167            -2.33   
4        4  oudtshoorn      ZA  -33.5907    22.2014            13.79   
5        5   tuatapere      NZ  -46.1333   167.6833            17.87   

   Humidity (%)  Cloudiness  Wind Speed  Datetime (Local)  
0          69.0        90.0        3.07      1.681604e+09  
2          74.0         6.0        6.04      1.681604e+09  
3          43.0         0.0        2.06      1.681604e+09  
4          56.0         6.0        0.65      1.681604e+09  
5          55.0        98.0        1.79      1.681604e+09

### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_condition.copy().loc[:, ["City", "Country","Latitude", "Longitude","Humidity (%)"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

City Country  Latitude  Longitude  Humidity (%) Hotel Name
0     hithadhoo      MV   -0.6000    73.0833          69.0           
2        atuona      PF   -9.8000  -139.0333          74.0           
3     rovaniemi      FI   66.5000    25.7167          43.0           
4    oudtshoorn      ZA  -33.5907    22.2014          56.0           
5     tuatapere      NZ  -46.1333   167.6833          55.0           
..          ...     ...       ...        ...           ...        ...
634      rawson      AR  -43.3002   -65.1023          57.0           
635     damghan      IR   36.1683    54.3480          41.0           
636  launceston      AU  -41.4500   147.1667          90.0           
637    sisimiut      GL   66.9395   -53.6735          82.0           
638   abu dhabi      AE   24.4667    54.3667          70.0           

[594 rows x 6 columns]

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "lang":"en",
    "limit":20
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
#     print(response.url)      
    # Convert the API response to JSON format
    name_address =  response.json()
    

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")



Starting hotel search
hithadhoo - nearest hotel: Pebbles Inn
atuona - nearest hotel: Pearl Resort
rovaniemi - nearest hotel: Matkustajakoti Outa
oudtshoorn - nearest hotel: The Pearl
tuatapere - nearest hotel: No hotel found
lanivtsi - nearest hotel: No hotel found
barrow - nearest hotel: No hotel found
san estanislao - nearest hotel: No hotel found
cape town - nearest hotel: Townhouse Hotel
albany - nearest hotel: No hotel found
ahipara - nearest hotel: YHA Ahipara
mataura - nearest hotel: No hotel found
busselton - nearest hotel: Broadwater Beach Resort
kapaa - nearest hotel: Pono Kai Resort
bengkulu - nearest hotel: Dena Hotel
padang - nearest hotel: losmen Surya
mogok - nearest hotel: Mogok Hill
faanui - nearest hotel: Oa Oa Lodge
provideniya - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
tayzhina - nearest hotel: No hotel found
ancud - nearest hotel: Cabañas
souillac - nearest hotel: Hotel SkylineInn
grindavik - nearest hotel: The Retreat at 

turukhansk - nearest hotel: Гостиница ОАО "Туруханскэнерго"
srandakan - nearest hotel: Puri Ganjuran
neuquen - nearest hotel: Hotel Argentina 307
huarmey - nearest hotel: Hostal Santa Rosa
hilo - nearest hotel: Dolphin Bay Hotel
richards bay - nearest hotel: Protea Hotel
ludvika - nearest hotel: X Hotell
tchollire - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
arinos - nearest hotel: No hotel found
tarakan - nearest hotel: Hotel Dynasty
imbituba - nearest hotel: Silvestre Praia Hotel
xichang - nearest hotel: No hotel found
uusikaupunki - nearest hotel: Gasthaus Pooki
hasaki - nearest hotel: 大新旅館
fairbanks - nearest hotel: Westmark Fairbanks Hotel
cidreira - nearest hotel: Hotel Castelo
sinnamary - nearest hotel: Hôtel du fleuve
discovery bay - nearest hotel: Holiday Inn Express & Suites Brentwood
severo-kurilsk - nearest hotel: "Дом рыбака"
broken hill - nearest hotel: Ibis Styles
srivardhan - nearest hotel: No hotel found
kaina - nearest hotel: 

southbridge - nearest hotel: Hamilton Inn Sturbridge
maniitsoq - nearest hotel: Hotel Heilmann Lyberth
thinadhoo - nearest hotel: No hotel found
aktash - nearest hotel: Акташ (Стонт)
muli - nearest hotel: Mulaku
mapiripan - nearest hotel: No hotel found
puerto madryn - nearest hotel: Hotel Gran Palace
bambanglipuro - nearest hotel: Puri Ganjuran
mabaruma - nearest hotel: Monty Brooms Hotel
bud - nearest hotel: No hotel found
pathein - nearest hotel: Taan Taan Ta
severnoye - nearest hotel: No hotel found
vrangel - nearest hotel: No hotel found
port hawkesbury - nearest hotel: Hearthstone Inn Port Hawkesbury
bagdarin - nearest hotel: No hotel found
freeport - nearest hotel: The Freeport Inn and Marina
pisco - nearest hotel: La Portada
bongor - nearest hotel: Hôtel Confort Annexe Bongor
maroantsetra - nearest hotel: Coco Beach
saint-joseph - nearest hotel: PALM Hotel & Spa
gambela - nearest hotel: Grand Hotel
hima - nearest hotel: No hotel found
dudinka - nearest hotel: Виктория
sosnovo-o

meulaboh - nearest hotel: Hotel Meuligou
fare - nearest hotel: hôtel Maitai Lapita Village
labrea - nearest hotel: No hotel found
mporokoso - nearest hotel: No hotel found
lugazi - nearest hotel: No hotel found
ginda - nearest hotel: Shivalinga
baykit - nearest hotel: No hotel found
taltal - nearest hotel: Hostería Taltal
san jose - nearest hotel: Hotel De Anza
yar-sale - nearest hotel: ЯЛЭМД
farap - nearest hotel: Amyderýa Myhmanhanasy
kefamenanu - nearest hotel: Hotel Setangkai
sao felix do xingu - nearest hotel: Hotel Terraço
campo verde - nearest hotel: No hotel found
madawaska - nearest hotel: Sheraton
marsa matruh - nearest hotel: صقور
rawson - nearest hotel: Hotel Deportivo
damghan - nearest hotel: هتل جهانگردى دامغان
launceston - nearest hotel: Hotel Charles
sisimiut - nearest hotel: Sømandshjemmet
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi


In [28]:
# Display sample data
hotel_df

City Country  Latitude  Longitude  Humidity (%)  \
0     hithadhoo      MV   -0.6000    73.0833          69.0   
2        atuona      PF   -9.8000  -139.0333          74.0   
3     rovaniemi      FI   66.5000    25.7167          43.0   
4    oudtshoorn      ZA  -33.5907    22.2014          56.0   
5     tuatapere      NZ  -46.1333   167.6833          55.0   
..          ...     ...       ...        ...           ...   
634      rawson      AR  -43.3002   -65.1023          57.0   
635     damghan      IR   36.1683    54.3480          41.0   
636  launceston      AU  -41.4500   147.1667          90.0   
637    sisimiut      GL   66.9395   -53.6735          82.0   
638   abu dhabi      AE   24.4667    54.3667          70.0   

                      Hotel Name  
0                    Pebbles Inn  
2                   Pearl Resort  
3            Matkustajakoti Outa  
4                      The Pearl  
5                 No hotel found  
..                           ...  
634              Hotel Deportivo  
635          هتل جهانگردى دامغان  
636                Hotel Charles  
637               Sømandshjemmet  
638  Marriott Downtown Abu-Dhabi  

[594 rows x 6 columns]

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
map_plot_2 = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
     geo = True,
    tiles = "OSM",
    size="Humidity (%)",
    color="Humidity (%)",
    cmap="rainbow",
    alpha=0.8,
    hover_cols=["City", "Country","Hotel Name", "Humidity"],
    width=800,
    height=600
)

# Display the map plot_1
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Humidity (%),City,Country,Hotel Name)